In [1]:
import cx_Oracle
import yaml
import pandas as pd
import logging

In [2]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.DEBUG, datefmt='%d-%b-%y %H:%M:%S')

In [3]:
def connect_to_mis_db(dsn,username,password):
    """ Connect to the database"""
    try:
        connection = cx_Oracle.connect(username, password, dsn)
        logging.info(f"Connection successful, connection version: {connection.version}")
        cursor = connection.cursor()
        cursor.arraysize = 2500
        return connection, cursor
    except cx_Oracle.DatabaseError as error_detail:
        logging.error('Connection failed due to:',error_detail)
        #raise

In [4]:
def query_mis_db(connection, cursor, query):
    logging.debug('Starting query to DB.')
    try:
        logging.debug('Executing query.')
        cursor.execute(query)
        allRows = []
        global names
        names = [ x[0] for x in cursor.description]
        for row in cursor.fetchall():               
            allRows.append(row)
        return pd.DataFrame(allRows, columns=names)
    finally:
        cursor.close()
        if connection is not None: 
            connection.close()

In [5]:
# load configs
def _read_yaml(config_path):
    with open(config_path, "r") as f:
        return yaml.safe_load(f)
    logging.info(f"Config file read in succesfully.")

In [6]:
def get_query(sql_query_path):
    sql_file = open(sql_query_path, "r")
    query = sql_file.read()
    logging.info(f'SQL query file read in successfully.')
    return query

In [7]:
def set_db_connection_config(credentials_path):
    credentials = _read_yaml(credentials_path)
    username = credentials['mis_db']['username']
    password = credentials['mis_db']['password']
    hostname = credentials['mis_db']['hostname']
    port = credentials['mis_db']['port']
    dsn = cx_Oracle.makedsn(hostname,port,'pmis01').replace('SID','SERVICE_NAME')
    logging.debug(f'DB Connection Config loaded successfully.')
    return dsn, username, password

In [8]:
def set_db_query_config(config_path):
    config = _read_yaml(config_path)
    credentials_path = config['mis_query']['credentials_path']
    sql_query_path = config['mis_query']['sql_query_path']
    raw_extract_path = config['mis_query']['raw_extract_path']
    raw_extract_filename = config['mis_query']['raw_extract_filename']
    logging.debug(f'DB Query Config loaded successfully.')
    return credentials_path, sql_query_path, raw_extract_path, raw_extract_filename

In [9]:
def main():
    config_path = '../conf/config.yml'
    credentials_path, sql_query_path, raw_extract_path, raw_extract_filename = set_db_query_config(config_path)
    dsn, username, password = set_db_connection_config(credentials_path)
    query = get_query(sql_query_path)
    connection, cursor = connect_to_mis_db(dsn,username,password)
    df = query_mis_db(connection, cursor, query)
    
    logging.debug('Writing out csv to disk.')
    df.to_csv(f'{raw_extract_path}{raw_extract_filename}', index=False)
    logging.info('Completed.')

In [ ]:
main()

25-Oct-21 16:05:26 - DB Query Config loaded successfully.
25-Oct-21 16:05:26 - DB Connection Config loaded successfully.
25-Oct-21 16:05:26 - SQL query file read in successfully.
25-Oct-21 16:05:27 - Connection successful, connection version: 11.2.0.4.0
25-Oct-21 16:05:27 - Starting query to DB.
25-Oct-21 16:05:27 - Executing query.
